In [6]:
import sys
sys.path.append('/opt/notebooks/polytrader')

In [7]:
from agents.polymarket.polymarket import Polymarket
from agents.trackers.market_tracker import OrderBook
from typing import Dict

In [8]:
p = Polymarket()

In [9]:
game = p.get_market_by_slug('nfl-phi-bal-2024-12-01')
outcome = game.outcomes[0]
# Initialize dictionary to store order books for each outcome
order_books: Dict[str, OrderBook] = {}

In [ ]:
def handle_market_update(outcome: str, message: dict):
    order_book = order_books.get(outcome)
    if order_book is None:
        order_book = OrderBook()
        order_books[outcome] = order_book
    
    if message['event_type'] == 'book':
        order_book.update_from_book(message)
    elif message['event_type'] == 'price_change':
        order_book.update_from_price_change(message)
    elif message['event_type'] == 'last_trade_price':
        order_book.update_last_trade(message)
    
    # Example of accessing historical data
    timestamps = order_book.get_timestamps()
    if timestamps:
        latest_ts = timestamps[-1]
        asks, bids = order_book.get_order_book_at_timestamp(latest_ts)
        print(f"Order book at {latest_ts}:")
        print("Asks:", asks)
        print("Bids:", bids)
        
        # Optionally clear old history to save memory
        if len(timestamps) > 1000:  # Keep last 1000 snapshots
            order_book.clear_history_before(timestamps[-1000])

In [ ]:
from agents.trackers.market_tracker import MarketChannel, MarketMessage
import asyncio
async def track(market_obj):
    # Initialize market channel
    market_channel = MarketChannel(market_obj)

    await market_channel.connect()

    # Add callback for specific assets
    market_channel.add_outcome_callback(outcome, handle_market_update)

    try:
        # Keep connection alive
        while True:
            await asyncio.sleep(1)
    except KeyboardInterrupt:
        await market_channel.close()

In [ ]:
await track(game)